In [23]:
import sagas.graph.dgraph_helper as helper
import pydgraph

def reset(schema):
    client = helper.create_client()
    # client.login("root", "password")
    helper.drop_all(client)
    helper.set_schema(client, schema)
    return client

client=reset('name: string @index(term) .')

In [24]:
txn = client.txn()
_ = txn.mutate(pydgraph.Mutation(commit_now=True), set_nquads="""
    <_:alice> <name> "Alice" .
    <_:greg> <name> "Greg" .
    <_:alice> <follows> <_:greg> .
""")

In [21]:
query = """query me($a: string) {
    me(func: anyofterms(name, "Alice"))
    {
        name
        follows
        {
            name
        }
    }
}"""
response = client.txn().query(query, variables={'$a': 'Alice'})
print(response.json)

rs=json.loads(response.json).get('me')
print(json.dumps(rs, ensure_ascii=False, indent=2))

b'{"me":[{"name":"Alice","follows":[{"name":"Greg"}]}]}'
[
  {
    "name": "Alice",
    "follows": [
      {
        "name": "Greg"
      }
    ]
  }
]


In [25]:
client=reset('''
    name: string @index(exact, term) .
    rated: uid @reverse @count .
''')

In [26]:
txn = client.txn()
_ = txn.mutate(pydgraph.Mutation(commit_now=True), set_nquads="""
    # -- Facets on scalar predicates
    _:alice <name> "Alice" .
    _:alice <mobile> "040123456" (since=2006-01-02T15:04:05) .
    _:alice <car> "MA0123" (since=2006-02-02T13:01:09, first=true) .

    _:bob <name> "Bob" .
    _:bob <car> "MA0134" (since=2006-02-02T13:01:09) .

    _:charlie <name> "Charlie" .
    _:dave <name> "Dave" .


    # -- Facets on UID predicates
    _:alice <friend> _:bob (close=true, relative=false) .
    _:alice <friend> _:charlie (close=false, relative=true) .
    _:alice <friend> _:dave (close=true, relative=true) .


    # -- Facets for variable propagation
    _:movie1 <name> "Movie 1" .
    _:movie2 <name> "Movie 2" .
    _:movie3 <name> "Movie 3" .

    _:alice <rated> _:movie1 (rating=3) .
    _:alice <rated> _:movie2 (rating=2) .
    _:alice <rated> _:movie3 (rating=5) .

    _:bob <rated> _:movie1 (rating=5) .
    _:bob <rated> _:movie2 (rating=5) .
    _:bob <rated> _:movie3 (rating=5) .

    _:charlie <rated> _:movie1 (rating=2) .
    _:charlie <rated> _:movie2 (rating=5) .
    _:charlie <rated> _:movie3 (rating=1) .
""")

In [29]:
response = client.txn().query('''{
  data(func: eq(name, "Alice")) {
     name
     mobile
     car
  }
}''')

def print_rs(response):
    rs=json.loads(response.json)
    print(json.dumps(rs, ensure_ascii=False, indent=2))

print_rs(response)

{
  "data": [
    {
      "name": "Alice",
      "mobile": "040123456",
      "car": "MA0123"
    }
  ]
}


In [30]:
response = client.txn().query('''{
  data(func: eq(name, "Alice")) {
     name
     mobile @facets(since)
     car @facets(since)
  }
}''')
print_rs(response)

{
  "data": [
    {
      "name": "Alice",
      "mobile|since": "2006-01-02T15:04:05Z",
      "mobile": "040123456",
      "car|since": "2006-02-02T13:01:09Z",
      "car": "MA0123"
    }
  ]
}


In [32]:
# 查询边缘上的所有方面@facets
response = client.txn().query('''{
  data(func: eq(name, "Alice")) {
     name
     mobile @facets
     car @facets
  }
}''')
print_rs(response)

{
  "data": [
    {
      "name": "Alice",
      "mobile|since": "2006-01-02T15:04:05Z",
      "mobile": "040123456",
      "car|first": true,
      "car|since": "2006-02-02T13:01:09Z",
      "car": "MA0123"
    }
  ]
}


在变异时，Facets键和值可以直接使用特定于语言的字符。但是<>在查询时需要将方面键括在尖括号中。这类似于谓词。

In [33]:
def set_nquads(nquads):
    txn = client.txn()
    return txn.mutate(pydgraph.Mutation(commit_now=True), set_nquads=nquads)

set_nquads("""
		_:person1 <name> "Daniel" (वंश="स्पेनी", ancestry="Español") .
		_:person2 <name> "Raj" (वंश="हिंदी", ancestry="हिंदी") .
		_:person3 <name> "Zhang Wei" (वंश="चीनी", ancestry="中文") .
""")

uids {
  key: "person1"
  value: "0x77ed"
}
uids {
  key: "person2"
  value: "0x77ee"
}
uids {
  key: "person3"
  value: "0x77ef"
}
context {
  start_ts: 30093
  commit_ts: 30094
  preds: "1-name"
  preds: "1-_predicate_"
}
latency {
  parsing_ns: 24721200
  processing_ns: 30775500
}

In [34]:
response = client.txn().query('''{
  q(func: has(name)) {
    name @facets(<वंश>)
  }
}''')
print_rs(response)

{
  "q": [
    {
      "name": "Movie 3"
    },
    {
      "name": "Alice"
    },
    {
      "name": "Bob"
    },
    {
      "name": "Charlie"
    },
    {
      "name": "Dave"
    },
    {
      "name": "Movie 1"
    },
    {
      "name": "Movie 2"
    },
    {
      "name|वंश": "स्पेनी",
      "name": "Daniel"
    },
    {
      "name|वंश": "हिंदी",
      "name": "Raj"
    },
    {
      "name|वंश": "चीनी",
      "name": "Zhang Wei"
    }
  ]
}


请求特定谓词时可以指定别名。语法类似于为其他谓词请求别名的方式。orderasc并且orderdesc不允许作为别名，因为它们具有特殊含义。除此之外，其他任何东西都可以设置为别名。
这里我们设置car_since，close_friend别名since，close分别刻面。

In [35]:
response = client.txn().query('''{
   data(func: eq(name, "Alice")) {
     name
     mobile
     car @facets(car_since: since)
     friend @facets(close_friend: close) {
       name
     }
   }
}''')
print_rs(response)

{
  "data": [
    {
      "name": "Alice",
      "mobile": "040123456",
      "car_since": "2006-02-02T13:01:09Z",
      "car": "MA0123",
      "friend": [
        {
          "name": "Bob",
          "close_friend": true
        },
        {
          "name": "Charlie",
          "close_friend": false
        },
        {
          "name": "Dave",
          "close_friend": true
        }
      ]
    }
  ]
}


In [36]:
def run_q(query):
    response = client.txn().query(query)
    print_rs(response)

run_q('''{
   data(func: eq(name, "Alice")) {
     name
     friend @facets(close) {
       name
     }
   }
}
''')

{
  "data": [
    {
      "name": "Alice",
      "friend": [
        {
          "name": "Bob",
          "friend|close": true
        },
        {
          "name": "Charlie",
          "friend|close": false
        },
        {
          "name": "Dave",
          "friend|close": true
        }
      ]
    }
  ]
}


In [37]:
run_q('''{
  data(func: eq(name, "Alice")) {
    name
    friend @facets {
      name
      car @facets
    }
  }
}
''')

{
  "data": [
    {
      "name": "Alice",
      "friend": [
        {
          "name": "Bob",
          "car|since": "2006-02-02T13:01:09Z",
          "car": "MA0134",
          "friend|close": true,
          "friend|relative": false
        },
        {
          "name": "Charlie",
          "friend|close": false,
          "friend|relative": true
        },
        {
          "name": "Dave",
          "friend|close": true,
          "friend|relative": true
        }
      ]
    }
  ]
}


In [38]:
# 基于facet过滤边缘
run_q('''{
  data(func: eq(name, "Alice")) {
    friend @facets(eq(close, true)) {
      name
    }
  }
}
''')

{
  "data": [
    {
      "friend": [
        {
          "name": "Bob"
        },
        {
          "name": "Dave"
        }
      ]
    }
  ]
}


In [39]:
# 要返回构面以及过滤器，请@facets(<facetname>)在查询中添加另一个构面
run_q('''{
  data(func: eq(name, "Alice")) {
    friend @facets(eq(close, true)) @facets(relative) { # filter close friends and give relative status
      name
    }
  }
}
''')

{
  "data": [
    {
      "friend": [
        {
          "name": "Bob",
          "friend|relative": false
        },
        {
          "name": "Dave",
          "friend|relative": true
        }
      ]
    }
  ]
}


In [40]:
# Facet查询可以用AND，OR和组成NOT
run_q('''{
  data(func: eq(name, "Alice")) {
    friend @facets(eq(close, true) AND eq(relative, true)) @facets(relative) { # filter close friends in my relation
      name
    }
  }
}
''')

{
  "data": [
    {
      "friend": [
        {
          "name": "Dave",
          "friend|relative": true
        }
      ]
    }
  ]
}


In [41]:
# 可以对uid边缘上的构面进行排序。在这里，我们对Alice，Bob和Charlie评定的电影进行排序，rating这是一个方面
run_q('''{
  me(func: anyofterms(name, "Alice Bob Charlie")) {
    name
    rated @facets(orderdesc: rating) {
      name
    }
  }
}
''')

{
  "me": [
    {
      "name": "Alice",
      "rated": [
        {
          "name": "Movie 3",
          "rated|rating": 5
        },
        {
          "name": "Movie 1",
          "rated|rating": 3
        },
        {
          "name": "Movie 2",
          "rated|rating": 2
        }
      ]
    },
    {
      "name": "Bob",
      "rated": [
        {
          "name": "Movie 3",
          "rated|rating": 5
        },
        {
          "name": "Movie 1",
          "rated|rating": 5
        },
        {
          "name": "Movie 2",
          "rated|rating": 5
        }
      ]
    },
    {
      "name": "Charlie",
      "rated": [
        {
          "name": "Movie 2",
          "rated|rating": 5
        },
        {
          "name": "Movie 1",
          "rated|rating": 2
        },
        {
          "name": "Movie 3",
          "rated|rating": 1
        }
      ]
    }
  ]
}


## 将Facet值分配给变量
UID边上的构面可以存储在值变量中。变量是从边缘目标到构面值的映射。
爱丽丝的朋友通过变量报告close和relative。

In [42]:
run_q('''{
  var(func: eq(name, "Alice")) {
    friend @facets(a as close, b as relative)
  }

  friend(func: uid(a)) {
    name
    val(a)
  }

  relative(func: uid(b)) {
    name
    val(b)
  }
}
''')

{
  "friend": [
    {
      "name": "Bob",
      "val(a)": true
    },
    {
      "name": "Charlie",
      "val(a)": false
    },
    {
      "name": "Dave",
      "val(a)": true
    }
  ],
  "relative": [
    {
      "name": "Bob",
      "val(b)": false
    },
    {
      "name": "Charlie",
      "val(b)": true
    },
    {
      "name": "Dave",
      "val(b)": true
    }
  ]
}


In [43]:
# UID边上的facet可以以边到facet值的映射的格式保存到值变量里
run_q('''{
  var(func: eq(name, "Alice")) {
    friend @facets(a as close, b as relative)
  }

  friend(func: uid(a)) {
    name
    val(a)
  }

  relative(func: uid(b)) {
    name
    val(b)
  }
}
''')

{
  "friend": [
    {
      "name": "Bob",
      "val(a)": true
    },
    {
      "name": "Charlie",
      "val(a)": false
    },
    {
      "name": "Dave",
      "val(a)": true
    }
  ],
  "relative": [
    {
      "name": "Bob",
      "val(b)": false
    },
    {
      "name": "Charlie",
      "val(b)": true
    },
    {
      "name": "Dave",
      "val(b)": true
    }
  ]
}


## 方面和可变传播
可以将face的值int和值float分配给变量，从而传播值。
爱丽丝，鲍勃和查理各自评价每部电影。facet rating上的值变量将电影映射到评级。通过多条路径到达电影的查询会对每条路径上的评级进行求和。以下总结了爱丽丝，鲍勃和查理对三部电影的收视率。

In [44]:
run_q('''{
  var(func: anyofterms(name, "Alice Bob Charlie")) {
    num_raters as math(1)
    rated @facets(r as rating) {
      total_rating as math(r) # sum of the 3 ratings
      average_rating as math(total_rating / num_raters)
    }
  }
  data(func: uid(total_rating)) {
    name
    val(total_rating)
    val(average_rating)
  }

}
''')

{
  "data": [
    {
      "name": "Movie 3",
      "val(total_rating)": 11,
      "val(average_rating)": 3.666667
    },
    {
      "name": "Movie 1",
      "val(total_rating)": 10,
      "val(average_rating)": 3.333333
    },
    {
      "name": "Movie 2",
      "val(total_rating)": 12,
      "val(average_rating)": 4.0
    }
  ]
}


In [45]:
## 可以聚合分配给值变量的构面值。
run_q('''{
  data(func: eq(name, "Alice")) {
    name
    rated @facets(r as rating) {
      name
    }
    avg(val(r))
  }
}
''')

{
  "data": [
    {
      "name": "Alice",
      "rated": [
        {
          "name": "Movie 3",
          "rated|rating": 5
        },
        {
          "name": "Movie 1",
          "rated|rating": 3
        },
        {
          "name": "Movie 2",
          "rated|rating": 2
        }
      ],
      "avg(val(r))": 3.333333
    }
  ]
}


In [47]:
# r是从电影到达到电影的查询中边缘的评级总和的地图。
# 因此，以下内容并未正确计算Alice和Bob的平均评分 - 
# 它计算的是Alice和Bob评级的平均值的2倍
run_q('''{
  data(func: anyofterms(name, "Alice Bob")) {
    name
    rated @facets(r as rating) {
      name
    }
    avg(val(r))
  }
}
''')

{
  "data": [
    {
      "name": "Alice",
      "rated": [
        {
          "name": "Movie 3",
          "rated|rating": 5
        },
        {
          "name": "Movie 1",
          "rated|rating": 3
        },
        {
          "name": "Movie 2",
          "rated|rating": 2
        }
      ],
      "avg(val(r))": 8.333333
    },
    {
      "name": "Bob",
      "rated": [
        {
          "name": "Movie 3",
          "rated|rating": 5
        },
        {
          "name": "Movie 1",
          "rated|rating": 5
        },
        {
          "name": "Movie 2",
          "rated|rating": 5
        }
      ],
      "avg(val(r))": 8.333333
    }
  ]
}


In [48]:
# 计算用户的平均评级需要一个将用户映射到其评级总和的变量
run_q('''{
  var(func: has(rated)) {
    num_rated as math(1)
    rated @facets(r as rating) {
      avg_rating as math(r / num_rated)
    }
  }

  data(func: uid(avg_rating)) {
    name
    val(avg_rating)
  }
}
''')

{
  "data": [
    {
      "name": "Movie 3",
      "val(avg_rating)": 3.666667
    },
    {
      "name": "Movie 1",
      "val(avg_rating)": 3.333333
    },
    {
      "name": "Movie 2",
      "val(avg_rating)": 4.0
    }
  ]
}
